In [ ]:
# Copyright 2024 Google LLC.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI SDK for Python: AutoML training tabular regression model for batch prediction

## Overview


This tutorial demonstrates how to use the Vertex AI SDK for Python to create tabular regression models and generate batch prediction using a Google Cloud [AutoML](https://cloud.google.com/vertex-ai/docs/start/automl-users) model.

Learn more about [Regression for tabular data](https://cloud.google.com/vertex-ai/docs/tabular-data/classification-regression/overview).

### Objective

In this tutorial, you learn how to create an AutoML tabular regression model and deploy it for batch prediction using the Vertex AI SDK for Python. You can alternatively create and deploy models using the `gcloud` command-line tool or batch using the Cloud Console.

This tutorial uses the following Google Cloud ML services:

- Vertex AI Datasets (Tabular)
- Vertex AI Training (AutoML Tabular Training)
- Vertex AI Model Registry
- Vertex AI batch prediction

The steps performed include:

- Create a Vertex AI dataset resource.
- Train an AutoML tabular regression model resource.
- Obtain the evaluation metrics for the model resource.
- Make a batch prediction.

### Dataset

The dataset used for this tutorial is the [GSOD dataset](https://console.cloud.google.com/marketplace/product/noaa-public/gsod) from [BigQuery public datasets](https://cloud.google.com/bigquery/public-data). In this version of the dataset, you use the year, month, and day fields to predict the mean daily temperature (`mean_temp`).

### Costs

This tutorial uses billable components of Google Cloud:

* Vertex AI
* BigQuery / BigQuery ML

Learn about [Vertex AI
pricing](https://cloud.google.com/vertex-ai/pricing), [Cloud Storage
pricing](https://cloud.google.com/storage/pricing) and [BigQuery pricing](https://cloud.google.com/bigquery/pricing) and use the [Pricing Calculator](https://cloud.google.com/products/calculator/) to generate a cost estimate based on your projected usage.

## Get started

### Install Vertex AI SDK for Python and other required packages


In [1]:
# Install the packages
! pip3 install --upgrade --quiet google-cloud-aiplatform \
                                    'google-cloud-bigquery[bqstorage,pandas]' \
                                    google-cloud-storage


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 56.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.5/130.5 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.1/239.1 kB 21.6 MB/s eta 0:00:00


### Restart runtime (Colab only)

To use the newly installed packages, you must restart the runtime on Google Colab.

In [2]:
import sys

if "google.colab" in sys.modules:

    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. Wait until it's finished before continuing to the next step. ⚠️</b>
</div>


### Authenticate your notebook environment (Colab only)

Authenticate your environment on Google Colab.


In [1]:
import sys

if "google.colab" in sys.modules:

    from google.colab import auth

    auth.authenticate_user()

### Set Google Cloud project information

To get started using Vertex AI, you must have an existing Google Cloud project. Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [2]:
PROJECT_ID = "jc-gcp-project"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

Updated property [core/project].


### Set up variables

Next, set up some variables used throughout the tutorial.
### Import libraries and define constants

In [3]:
from google.cloud import aiplatform, bigquery

## Initialize Vertex AI SDK for Python

Initialize the Vertex AI SDK for Python for your project and the corresponding bucket.

In [4]:
aiplatform.init(project=PROJECT_ID, location=LOCATION)

## Tutorial

Now you're ready to start creating your own AutoML tabular regression model.

### Location of BigQuery training data.

Set the `IMPORT_File` variable to the location of the data table in BigQuery.

In [5]:
IMPORT_FILE = "bigquery-public-data.samples.gsod"

#### Prepare the batch prediction data

Create two datasets from the original data.

In [6]:
# Create client in default location
bq_client = bigquery.Client(
    project=PROJECT_ID,
    credentials=aiplatform.initializer.global_config.credentials,
)

In [10]:
%%bigquery
select * from `bigquery-public-data.samples.gsod` limit 10

Query is running:   0%|          |

Downloading:   0%|          |

,station_number,wban_number,year,month,day,mean_temp,num_mean_temp_samples,mean_dew_point,num_mean_dew_point_samples,mean_sealevel_pressure,...,min_temperature,min_temperature_explicit,total_precipitation,snow_depth,fog,rain,snow,hail,thunder,tornado
0,30750,99999,1929,10,5,42.000000,4,36.500000,4,998.900024,...,NaN,<NA>,NaN,NaN,False,False,False,False,False,False
1,38110,99999,1929,12,20,45.500000,4,41.299999,4,1011.500000,...,NaN,<NA>,NaN,NaN,False,False,False,False,False,False
2,30750,99999,1929,10,14,52.500000,4,48.000000,4,1012.799988,...,NaN,<NA>,NaN,NaN,False,False,False,False,False,False
3,33790,99999,1929,11,14,32.799999,4,30.200001,4,1001.000000,...,NaN,<NA>,0.0,NaN,True,True,True,True,True,True
4,37950,99999,1929,11,12,45.200001,4,39.000000,4,994.500000,...,NaN,<NA>,NaN,NaN,False,False,False,False,False,False
5,33790,99999,1929,11,9,40.200001,4,34.500000,4,1012.200012,...,NaN,<NA>,0.0,NaN,False,False,False,False,False,False
6,37950,99999,1929,12,4,49.700001,4,47.799999,4,1003.099976,...,NaN,<NA>,NaN,NaN,False,False,False,False,False,False
7,37770,99999,1929,8,20,59.700001,4,49.200001,4,1022.299988,...,NaN,<NA>,0.0,NaN,False,False,False,False,False,False
8,37950,99999,1929,12,13,48.299999,4,46.000000,4,1026.199951,...,NaN,<NA>,0.0,NaN,False,False,False,False,False,False
9,30750,99999,1929,11,16,34.200001,4,31.500000,4,998.299988,...,NaN,<NA>,0.0,NaN,False,False,False,False,False,False


In [7]:
# Create training dataset in default location
TRAINING_INPUT_DATASET_ID = "gsod_training_unique"
bq_dataset = bigquery.Dataset(f"{PROJECT_ID}.{TRAINING_INPUT_DATASET_ID}")
bq_dataset = bq_client.create_dataset(bq_dataset)
print(f"Created dataset {bq_client.project}.{bq_dataset.dataset_id}")

# Create test dataset in default location
PREDICTION_INPUT_DATASET_ID = "gsod_prediction_unique"
bq_dataset = bigquery.Dataset(f"{PROJECT_ID}.{PREDICTION_INPUT_DATASET_ID}")
bq_dataset = bq_client.create_dataset(bq_dataset)
print(f"Created dataset {bq_client.project}.{bq_dataset.dataset_id}")

Created dataset jc-gcp-project.gsod_training_unique
Created dataset jc-gcp-project.gsod_prediction_unique


In [11]:
# Select top 3000 rows of dataset
TRAINING_SIZE = 3000
query = f"""
        SELECT *
        FROM {IMPORT_FILE}
        LIMIT {TRAINING_SIZE}
        """

TRAINING_INPUT_TABLE_ID = f"{PROJECT_ID}.{TRAINING_INPUT_DATASET_ID}.test"
job_config = bigquery.QueryJobConfig(destination=TRAINING_INPUT_TABLE_ID)

query_job = bq_client.query(query, job_config=job_config)  # API request
query_job.result()  # Waits for query to finish

In [12]:
# Select a subset of the original dataset for testing
PREDICTION_SIZE = 100
query = f"""
        SELECT *
        FROM {IMPORT_FILE}
        LIMIT {PREDICTION_SIZE}
        OFFSET {TRAINING_SIZE}
        """

PREDICTION_INPUT_TABLE_ID = f"{PROJECT_ID}.{PREDICTION_INPUT_DATASET_ID}.prediction"
job_config = bigquery.QueryJobConfig(destination=PREDICTION_INPUT_TABLE_ID)

query_job = bq_client.query(query, job_config=job_config)  # API request
query_job.result()  # Waits for query to finish

### Create the Dataset

Use `TabularDataset.create()` method to create a tabular dataset resource, which takes the following parameters:

- `display_name`: The human readable name for the dataset resource.
- `gcs_source`: A list of one or more dataset index files to import the data items into the dataset resource.
- `bq_source`: Alternatively, import data items from a BigQuery table into the dataset resource.

This operation may take several minutes.

In [14]:
dataset = aiplatform.TabularDataset.create(
    display_name="NOAA historical weather data_unique",
    bq_source=[f"bq://{TRAINING_INPUT_TABLE_ID}"],
)

label_column = "mean_temp"

print(dataset.resource_name)

INFO:google.cloud.aiplatform.datasets.dataset:Creating TabularDataset
INFO:google.cloud.aiplatform.datasets.dataset:Create TabularDataset backing LRO: projects/78565683329/locations/us-central1/datasets/528030013878960128/operations/7164820909398687744
INFO:google.cloud.aiplatform.datasets.dataset:TabularDataset created. Resource name: projects/78565683329/locations/us-central1/datasets/528030013878960128
INFO:google.cloud.aiplatform.datasets.dataset:To use this TabularDataset in another session:
INFO:google.cloud.aiplatform.datasets.dataset:ds = aiplatform.TabularDataset('projects/78565683329/locations/us-central1/datasets/528030013878960128')


projects/78565683329/locations/us-central1/datasets/528030013878960128


In [16]:
COLUMN_SPECS = {
    "year": "auto",
    "month": "auto",
    "day": "auto",
}

label_column = "mean_temp"

### Create and run training pipeline

To train an AutoML model, create and run a training pipeline.

#### Create training pipeline

Create an AutoML training pipeline using the `AutoMLTabularTrainingJob` class, with the following parameters:

- `display_name`: The human readable name for the training job resource.
- `optimization_prediction_type`: The type task to train the model for.
  - `classification`: A tabular classification model.
  - `regression`: A tabular regression model.
- `column_transformations`: (Optional): Transformations to apply to the input columns.
- `optimization_objective`: The optimization objective (minimize or maximize).
  - binary classification:
    - `minimize-log-loss`
    - `maximize-au-roc`
    - `maximize-au-prc`
    - `maximize-precision-at-recall`
    - `maximize-recall-at-precision`
  - multi-class classification:
    - `minimize-log-loss`
  - regression:
    - `minimize-rmse`
    - `minimize-mae`
    - `minimize-rmsle`

The instantiated object is the DAG (directed acyclic graph) for the training pipeline.

In [17]:
training_job = aiplatform.AutoMLTabularTrainingJob(
    display_name="job_unique",
    optimization_prediction_type="regression",
    optimization_objective="minimize-rmse",
    column_specs=COLUMN_SPECS,
)

print(training_job)

#### Run the training pipeline

Run the training job by invoking the `run` method with the following parameters:

- `dataset`: The dataset resource to train the model.
- `model_display_name`: The human readable name for the trained model.
- `training_fraction_split`: The percentage of the dataset to use for training.
- `test_fraction_split`: The percentage of the dataset to use for test (holdout data).
- `validation_fraction_split`: The percentage of the dataset to use for validation.
- `target_column`: The name of the column to train as the label.
- `budget_milli_node_hours`: (optional) Maximum training time specified in unit of millihours (1000 = hour).
- `disable_early_stopping`: By default, the model training stops early if the model performance doesn't improve. Setting `disable_early_stopping` = `True` overrides this behavior, allowing the model to train for the entire specified duration.

The `run` method, upon completion, returns the model resource.

The execution of the training pipeline may take upto 3 hours.

In [18]:
model = training_job.run(
    dataset=dataset,
    model_display_name="model_unique",
    training_fraction_split=0.6,
    validation_fraction_split=0.2,
    test_fraction_split=0.2,
    budget_milli_node_hours=1000,
    disable_early_stopping=False,
    target_column=label_column,
)

INFO:google.cloud.aiplatform.training_jobs:View Training:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/8292605035573936128?project=78565683329
INFO:google.cloud.aiplatform.training_jobs:AutoMLTabularTrainingJob projects/78565683329/locations/us-central1/trainingPipelines/8292605035573936128 current state:
PipelineState.PIPELINE_STATE_RUNNING
INFO:google.cloud.aiplatform.training_jobs:AutoMLTabularTrainingJob projects/78565683329/locations/us-central1/trainingPipelines/8292605035573936128 current state:
PipelineState.PIPELINE_STATE_RUNNING
INFO:google.cloud.aiplatform.training_jobs:AutoMLTabularTrainingJob projects/78565683329/locations/us-central1/trainingPipelines/8292605035573936128 current state:
PipelineState.PIPELINE_STATE_RUNNING
INFO:google.cloud.aiplatform.training_jobs:AutoMLTabularTrainingJob projects/78565683329/locations/us-central1/trainingPipelines/8292605035573936128 current state:
PipelineState.PIPELINE_STATE_RUNNING
INFO:google.cloud.aipl

## Review model evaluation scores
After model training is complete, you can review its evaluation scores.

In [19]:
# Get evaluations
model_evaluations = model.list_model_evaluations()

model_evaluation = list(model_evaluations)[0]
print(model_evaluation)

resource name: projects/78565683329/locations/us-central1/models/3714138183999422464@1/evaluations/5004591682340601851


## Send a batch prediction request

Now you can make a batch prediction.

### Create a results dataset

Create a dataset to store the prediction results.

In [25]:
# Create results dataset in default location
RESULTS_DATASET_ID = "gsod_results_unique"
bq_dataset = bigquery.Dataset(f"{PROJECT_ID}.{RESULTS_DATASET_ID}")
bq_dataset = bq_client.create_dataset(bq_dataset)
print(f"Created dataset {bq_client.project}.{bq_dataset.dataset_id}")

Created dataset jc-gcp-project.gsod_results_unique


### Make the batch prediction request

You can make a batch prediction by invoking the `batch_predict()` method, with the following parameters:

- `job_display_name`: The human readable name for the batch prediction job.
- `gcs_source`: A list of one or more batch request input files.
- `gcs_destination_prefix`: The Cloud Storage location for storing the batch prediction resuls.
- `instances_format`: The format for the input instances, either 'bigquery', 'csv' or 'jsonl'. Defaults to 'jsonl'.
- `predictions_format`: The format for the output predictions, either 'csv' or 'jsonl'. Defaults to 'jsonl'.
- `machine_type`: The type of machine to use for training.
- `accelerator_type`: The hardware accelerator type.
- `accelerator_count`: The number of accelerators to attach to a worker replica.
- `sync`: Set `True` to wait until the completion of the job.

Batch prediction job takes roughly 1 hour to finish.

In [26]:
# Note: The bigquery_source and bigquery_destination_prefix must be in the same location
PREDICTION_RESULTS_DATASET_ID = f"{PROJECT_ID}.{RESULTS_DATASET_ID}"

batch_predict_job = model.batch_predict(
    job_display_name="tabular_regression_batch_predict_job",
    bigquery_source=f"bq://{PREDICTION_INPUT_TABLE_ID}",
    instances_format="bigquery",
    predictions_format="bigquery",
    bigquery_destination_prefix=f"bq://{PREDICTION_RESULTS_DATASET_ID}",
)

INFO:google.cloud.aiplatform.jobs:Creating BatchPredictionJob
INFO:google.cloud.aiplatform.jobs:BatchPredictionJob created. Resource name: projects/78565683329/locations/us-central1/batchPredictionJobs/2965550363836416000
INFO:google.cloud.aiplatform.jobs:To use this BatchPredictionJob in another session:
INFO:google.cloud.aiplatform.jobs:bpj = aiplatform.BatchPredictionJob('projects/78565683329/locations/us-central1/batchPredictionJobs/2965550363836416000')
INFO:google.cloud.aiplatform.jobs:View Batch Prediction Job:
https://console.cloud.google.com/ai/platform/locations/us-central1/batch-predictions/2965550363836416000?project=78565683329
INFO:google.cloud.aiplatform.jobs:BatchPredictionJob projects/78565683329/locations/us-central1/batchPredictionJobs/2965550363836416000 current state:
JobState.JOB_STATE_PENDING
INFO:google.cloud.aiplatform.jobs:BatchPredictionJob projects/78565683329/locations/us-central1/batchPredictionJobs/2965550363836416000 current state:
JobState.JOB_STATE_PEN

###  View the batch prediction results

Use the BigQuery Python client to query the destination table and return results as a Pandas dataframe.

In [27]:
dataframe = (
    bq_client.query(f"SELECT * FROM `{PREDICTION_RESULTS_DATASET_ID}.*`")
    .result()
    .to_dataframe()
)

print(dataframe.head())

  day    fog   hail  max_gust_wind_speed  max_sustained_wind_speed  \
0   9  False  False                  NaN                      18.1   
1  14  False  False                  NaN                      13.0   
2  28  False  False                  NaN                      13.0   
3   8  False  False                  NaN                      13.0   
4  30  False  False                  NaN                       8.9   

   max_temperature  max_temperature_explicit  mean_dew_point  \
0             52.0                     False             NaN   
1             55.0                     False             NaN   
2             50.0                     False       49.000000   
3             32.0                     False       36.500000   
4             55.0                     False       56.200001   

   mean_sealevel_pressure  mean_station_pressure  ...  \
0             1017.700012                    NaN  ...   
1             1012.000000                    NaN  ...   
2             1012.0999

# Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial:

- Model
- AutoML Training Job
- Batch Job

In [ ]:
# Delete BigQuery datasets
bq_client.delete_dataset(
    f"{PROJECT_ID}.{TRAINING_INPUT_DATASET_ID}",
    delete_contents=True,
    not_found_ok=True,
)

bq_client.delete_dataset(
    f"{PROJECT_ID}.{PREDICTION_INPUT_DATASET_ID}",
    delete_contents=True,
    not_found_ok=True,
)

bq_client.delete_dataset(
    f"{PROJECT_ID}.{RESULTS_DATASET_ID}", delete_contents=True, not_found_ok=True
)

# Delete Vertex AI resources
dataset.delete()
model.delete()
training_job.delete()
batch_predict_job.delete()